# 웹 검색

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
    topic="general",  # general/news/finance
    search_depth="basic",  # basic/advance
    # time_range="day",  # 데이터 최신 기준, None/day/week/month/year
    # include_domains=None,
    # exclude_domains=None  # 특정 도메인 포함/제외
)

search_tool.invoke({'query': '서울고속버스터미널 근처에서, 제로페이가 되는 점심 먹을 만한 곳을 알려줘'})

{'query': '서울고속버스터미널 근처에서, 제로페이가 되는 점심 먹을 만한 곳을 알려줘',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.diningcode.com/list.dc?query=%EA%B3%A0%EC%86%8D%ED%84%B0%EB%AF%B8%EB%84%90%20%EC%A0%9C%EB%A1%9C%ED%8E%98%EC%9D%B4',
   'title': "'고속터미널 제로페이' 맛집 빅데이터 추천순위 Top3",
   'content': '고속터미널 제로페이맛집 (3곳) · 1. 이디야커피 서울고속버스터미널점 · 2. 이디야 반포지하상가점 · 3. 써브웨이 서울고속터미널점.',
   'score': 0.7961819,
   'raw_content': None},
  {'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EA%B3%A0%EC%86%8D%EB%B2%84%EC%8A%A4%ED%84%B0%EB%AF%B8%EB%84%90%20%EC%A0%9C%EB%A1%9C%ED%8E%98%EC%9D%B4',
   'title': "'강남고속버스터미널 제로페이' 맛집 빅데이터 추천순위 Top3",
   'content': '강남고속버스터미널 제로페이맛집 (3곳) · 1. 이디야커피 서울고속버스터미널점 · 2. 이디야 반포지하상가점 · 3. 써브웨이 서울고속터미널점.',
   'score': 0.79293,
   'raw_content': None},
  {'url': 'https://www.tripadvisor.co.kr/RestaurantsNear-g294197-d9579467-Express_Bus_Terminal-Seoul.html',
   'title': '서울 고속 버스 터미널 역 근처 맛집 Best 10 - Tripadvisor',
   'content

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template='''
너는 웹 검색결과를 요약해서, 질문에 맞게 잘 정리해서 좋은 답변을 주는 챗봇이야.
---
질문: {query}
---
검색결과: {search_results}
---
질문에 맞는 답을 검색결과 바탕으로 잘 정리해서 알려줘.
'''
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 RunnablePassthrough() 위치에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']}: {r['content']}' for r in x['results']]
            )
        )
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

res = chain.invoke('서울고속터미널에서 점심먹기 좋은 곳을 알려줘. 그런데, 제로페이나 비플페이가 가능한 곳만 알려줘')

print(res)

검색 결과를 통해 서울고속터미널(서울터미널역) 주변에서 제로페이 또는 비플페이로 결제할 수 있는 점심 맛집 정보를 찾기 어렵습니다. 현재 제공된 자료에는 해당 결제 방식을 지원하는 구체적인 식당 목록이나 추천은 나타나지 않고 있습니다.

하지만 일반적으로 서울고속터미널 주변에는 가격대가 다양하고 제로페이, 비플페이 결제 방식을 도입한 인기 식당들이 있을 가능성이 있으니, 터미널 내 또는 인근의 제로페이 가맹점을 미리 확인하거나, 방문 전에 해당 가맹점의 결제 수단 안내를 참고하는 것도 좋은 방법입니다.

혹시 더 구체적인 조건이나 특정한 음식 종류를 원하신다면 알려 주세요!


## agent

1. 쿼리를 보고 계획을 짬
2. 필요한 검색을 수행(tools 사용)
3. 판단 후 결과 반환

In [ ]:
# 단, 이제 위에 있던 복잡한 코드를 볼 일은 없을 것이다. agent 기능에 통합됨

from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime

today = datetime.today().strftime('%D')

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 AI 어시스턴트야. 너는 반드시 웹 검색도구를 사용해서 정보를 얻어야 해. 가장 의미있는 검색결과들을 요약해서 질문에 답해줘. 참고로, 오늘 날짜는 미국식 날짜 표기로 {today}야.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt,
)

agent_executer = AgentExecutor(agent=agent, memory=memory, tools=[search_tool], verbose=True)

In [36]:
agent_executer.invoke({'input': '가게 이름만 알려줘'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '고속터미널역 근처 맛집'}`


{'query': '고속터미널역 근처 맛집', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://app.catchtable.co.kr/ct/map/COMMON?showTabs=true&serviceType=INTEGRATION&keyword=%EA%B3%A0%EC%86%8D%ED%84%B0%EB%AF%B8%EB%84%90%EC%97%AD%20%EC%B6%94%EC%B2%9C%20%EB%A7%9B%EC%A7%91&keywordSearch=%EA%B3%A0%EC%86%8D%ED%84%B0%EB%AF%B8%EB%84%90%EC%97%AD%20%EC%B6%94%EC%B2%9C%20%EB%A7%9B%EC%A7%91&bottomSheetHeightType=HALF', 'title': '고속터미널역 추천 맛집 예약하기 | 즐거운 미식생활의 시작, 캐치테이블', 'content': '고속터미널역 추천 맛집 예약하기 | 즐거운 미식생활의 시작', 'score': 0.98525, 'raw_content': None}, {'url': 'https://m.blog.naver.com/unisonplay/223049115891', 'title': '[고속터미널역] 센트럴시티터미널 맛집 총정리 : 네이버 블로그 - NAVER', 'content': '기본 짜장면 한 그릇 6,500 원 주관적으로 생각건대 찬이 잘 나오는 건 아닌듯 하다 3 ~ 4 인 한 상이 1 인분 요리에 비해 훨씬 맛이 좋다 본질적으로도 조리방식이 다른 것을 알 수 있다 딸려 나오는 선짓국과 게장, 막국수의 구성 또한 좋다 안주를 여러 개 시켜놓고 먹을 수 없다는 점 정도 다시 한 번 찾을만 하다 비교적 저렴한 가격으로 pot을 교

{'input': '가게 이름만 알려줘',
 'chat_history': [HumanMessage(content='여긴 고속터미널역 근처인데, 뭔가 먹으러 갈 만한데가 있을까?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='고속터미널역 근처에는 다양한 맛집이 추천됩니다. 몇 가지 인기 있는 곳들로는:\n\n1. **캐치테이블에서 추천하는 맛집들**: 예약도 가능하며, 여러 종류의 맛집이 모여 있어 선택의 폭이 넓어요. [캐치테이블 링크](https://app.catchtable.co.kr/ct/map/COMMON?showTabs=true&serviceType=INTEGRATION&keyword=%EA%B3%A0%EC%86%8D%ED%84%B0%EB%AF%B8%EB%84%90%EC%97%AD%20%EC%B6%94%EC%B2%9C%20%EB%A7%9B%EC%A7%91)\n\n2. **센트럴시티터미널 맛집 총정리**: 저렴하고 맛있는 한 끼를 즐길 수 있는 곳들이 많으며, 조리방식이 다양한 식당들이 있습니다. 예를 들면, 강원도식 명태식해와 탄탄면이 인기입니다. [네이버 블로그 글](https://m.blog.naver.com/unisonplay/223049115891)\n\n3. **이탈리안 피자 전문점**: 한국과 이탈리아의 풍미를 느낄 수 있는 피자집도 추천받고 있어요. [블로그 포스트](https://blog.naver.com/redangel_seoul/223581850627)\n\n4. **베스트 맛집 리스트**: 차알 파미에스테이션점, 딤딤섬 센트럴시티점 등 다양한 식당들이 인기가 많습니다. [바다문의 리스트](https://jdblue2022.tistory.com/entry/%EC%84%9C%EC%9A%B8-%EA%B3%A0%EC%86%8D%ED%84%B0%EB%AF%B8%EB%84%90-%EB%A7%9B%EC%A7%91-%EB%B2%A0%EC%8A%A4%ED%8A%B810)\n